In [41]:
import requests
import json
import os
import pandas as pd
import math
import time

<h3>Start by getting your RIDB developer key and storing it as an enviroment variable</h3>

In [2]:
# lets check to make sure your RIDB Key was stored in as an enviroment variable
key = os.environ.get('RIDBKEY')
print(key)


dcFeqNe0qikWKvdgW5WudJWX7XPrCdssLvWMj7gj


In [3]:
# Configure API request

endpoint = "https://developer.nps.gov/api/v1/campgrounds"

params = {'api_key': key}

response = requests.get(endpoint, params=params)

In [4]:
response.request.url

'https://developer.nps.gov/api/v1/campgrounds?api_key=dcFeqNe0qikWKvdgW5WudJWX7XPrCdssLvWMj7gj'

In [5]:
response

<Response [200]>

In [ ]:
txt = json.loads(response.content)


In [ ]:
txt

In [50]:
ridbendpoint = "https://ridb.recreation.gov/api/v1/facilities"
ridbkey = '9404d711-544a-48e4-8e75-2b8c04d4b4eb'


ridbparams = {'limit':50,'offset':0,'apikey': ridbkey}

response = requests.get(ridbendpoint, params=ridbparams)

In [45]:
print(response.request.url, response.status_code)

https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=1&apikey=9404d711-544a-48e4-8e75-2b8c04d4b4eb 200


In [46]:
facilities2 = json.loads(response.content)
facilities2.keys()

dict_keys(['RECDATA', 'METADATA'])

In [27]:
facilities['METADATA']

{'RESULTS': {'CURRENT_COUNT': 50, 'TOTAL_COUNT': 14022},
 'SEARCH_PARAMETERS': {'QUERY': '', 'LIMIT': 50, 'OFFSET': 0}}

In [36]:
math.ceil(14022 / 50)

281

In [48]:
facilities['RECDATA'][1]

{'FacilityID': '249458',
 'LegacyFacilityID': '',
 'OrgFacilityID': '28970',
 'ParentOrgID': '131',
 'ParentRecAreaID': '2018',
 'FacilityName': 'Columbine Picnic Site',
 'FacilityDescription': '<p>This is a picnic ground\xa0and rest area along US Highway 160. No Bathrooms.</p>',
 'FacilityTypeDescription': 'Facility',
 'FacilityUseFeeDescription': '',
 'FacilityDirections': 'From the city of South Fork travel West on US Hwy 160 for 10 miles.  The Columbine Rest Area and interp site are on the right (north) side of the highway.',
 'FacilityPhone': '',
 'FacilityEmail': '',
 'FacilityReservationURL': '',
 'FacilityMapURL': '',
 'FacilityAdaAccess': '',
 'GEOJSON': {'TYPE': 'Point', 'COORDINATES': [-106.7572802, 37.57209691]},
 'FacilityLongitude': -106.7572802,
 'FacilityLatitude': 37.57209691,
 'Keywords': '',
 'StayLimit': '',
 'Reservable': False,
 'Enabled': True,
 'LastUpdatedDate': '2020-04-10'}

In [49]:
facilities2['RECDATA'][0]

{'FacilityID': '249458',
 'LegacyFacilityID': '',
 'OrgFacilityID': '28970',
 'ParentOrgID': '131',
 'ParentRecAreaID': '2018',
 'FacilityName': 'Columbine Picnic Site',
 'FacilityDescription': '<p>This is a picnic ground\xa0and rest area along US Highway 160. No Bathrooms.</p>',
 'FacilityTypeDescription': 'Facility',
 'FacilityUseFeeDescription': '',
 'FacilityDirections': 'From the city of South Fork travel West on US Hwy 160 for 10 miles.  The Columbine Rest Area and interp site are on the right (north) side of the highway.',
 'FacilityPhone': '',
 'FacilityEmail': '',
 'FacilityReservationURL': '',
 'FacilityMapURL': '',
 'FacilityAdaAccess': '',
 'GEOJSON': {'TYPE': 'Point', 'COORDINATES': [-106.7572802, 37.57209691]},
 'FacilityLongitude': -106.7572802,
 'FacilityLatitude': 37.57209691,
 'Keywords': '',
 'StayLimit': '',
 'Reservable': False,
 'Enabled': True,
 'LastUpdatedDate': '2020-04-10'}